In [1]:
import torch

In [1]:
import os

In [ ]:
import os

# Path to your image folder
folder_path = "../archive"

# Get all files in the folder
files = os.listdir(folder_path)

# Filter only image files (you can add more extensions if needed)
image_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]

# Sort (optional, for consistent order)
image_files.sort()

# Loop through and rename
for i, filename in enumerate(image_files, start=1):
    # Get the file extension
    print(i)
    ext = os.path.splitext(filename)[1]
    
    # Create new filename like image_1.jpg
    new_name = f"image_{i}{ext}"
    
    # Old and new paths
    old_path = os.path.join(folder_path, filename)
    new_path = os.path.join(folder_path, new_name)
    
    # Rename the file
    os.rename(old_path, new_path)

print("✅ All images renamed successfully!")


In [ ]:
import cv2
input_dir='../archive'
output_dir='../cropped'

os.makedirs(output_dir,exist_ok=True)

target_size =(512,512)

for file in os.listdir(input_dir):
    if file.lower().endswith(('.png','.jpg','.jpeg','.bmp','.tiff')):
        img_path=os.path.join(input_dir,file)
        img=cv2.imread(img_path)
        print(img_path)
        resized=cv2.resize(img,target_size)
        cv2.imwrite(os.path.join(output_dir,file),resized)
print('done')

In [4]:
import copy
import os
import random
import shutil
import zipfile
from math import atan2, cos, sin, sqrt, pi, log

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
from numpy import linalg as LA
from torch import optim, nn
from torch.utils.data import DataLoader, random_split
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from tqdm import tqdm

In [6]:
class DoubleConv(nn.Module):
    def __init__(self,in_chan,out_chan):
        super().__init__()
        self.conv_op=nn.Sequential(
            nn.Conv2d(in_chan,out_chan,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_chan,out_chan,kernel_size=3,padding=1),
            nn.ReLU(inplace=True)
        )
    def forward(self,x):
        return self.conv_op(x)

In [7]:
class Downsampler(nn.Module):
    def __init__(self,in_chan,out_chan):
        super().__init__()
        self.conv=DoubleConv(in_chan,out_chan)
        self.poll=nn.MaxPool2d(kernel_size=2,stride=2)
    def forward(self,x):
        down=self.conv(x)
        p=self.pool(down)
        return down , p

In [8]:
class Upsampler(nn.Module):
    def __init__(self,in_chan,out_chan):
        super().__init__()
        self.up=nn.ConvTranspose2d(in_chan,in_chan//2,kernel_size=2,stride=2)
        self.conv=DoubleConv(in_chan , out_chan)
    def forward(self,x1,x2):
        x1=self.up(x1)
        x=torch.cat([x1,x2],1)
        return self.conv(x)
    

In [ ]:
class UNet(nn.Module):
    def __init__(self,in_chan,num_classes):
        super().__init__()
        self.down_c1=Downsampler(in_channels,64)
        self.down_c2=Downsampler(64,128)
        self.down_c3=Downsampler(128,256)
        self.down_c4=Downsampler(256,512)

        self.bottle_neck=DoubleConv(512,1024)

        self.up_c1=Upsampler(1024,512)
        self.up_c2=Upsampler(512,256)
        self.up_c3=Upsampler(256,128)
        self.up_c4=Upsampler(128,64)

        self.out=nn.Conv2d(in_chan=64,out_)